## Basic concept
==Request headers== are sent by the client to the server and contain information and instructions related to the requested resource.==Response headers are opposite==
+ Accept:define the media type that the clien can accept from server 
  + Accept: application/json, text/html
+ User-Agent: which browser or app makes the requests
  + google, microsoft edge，python-requests/2.31.0'
+ Authorization:send the client’s credentials to the server
+ Content-Type:media type of the content in the request body.
  + Content-Type: application/json
+ cookie
+ Cache-Control
  + Cache-Control: max-age=3600, public . the client to cache the response for a maximum of 3600 seconds (1 hour) and allows caching by public caches.

In [22]:
import requests
from requests.exceptions import HTTPError

URLS = ["https://api.github.com","https://api.github.com/invalid"]
'''
If you invoke .raise_for_status(), then Requests will raise an HTTPError for status codes between 400 and 600. 
If the status code indicates a successful request, then the program will proceed without raising that exception.

'''
for url in URLS:
    try:
        response = requests.get(url)
        print("请求头内容：")
        print(response.request.headers)
        print("响应头内容：")
        print(response.headers)
        print(response.raise_for_status()) # if success, it's NONE
        print(response.content) #<class 'bytes'>
        response.text # convert to string
        response_dict =response.json() # json is one dict. {key:value,key:value,key:value}

    except HTTPError as http_err: 
        print(f"HTTP error occurred: {http_err}") # 404
    except Exception as err:
        print(f"Other error occurred: {err}")
    else:
        print("Success!")

{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}
响应头内容：
{'Server': 'GitHub.com', 'Date': 'Mon, 03 Jun 2024 03:57:56 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Cache-Control': 'public, max-age=60, s-maxage=60', 'Vary': 'Accept, Accept-Encoding, Accept, X-Requested-With', 'ETag': 'W/"4f825cc84e1c733059d46e76e6df9db557ae5254f9625dfe8e1b09499c449438"', 'X-GitHub-Media-Type': 'github.v3; format=json', 'x-github-api-version-selected': '2022-11-28', 'Access-Control-Expose-Headers': 'ETag, Link, Location, Retry-After, X-GitHub-OTP, X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Used, X-RateLimit-Resource, X-RateLimit-Reset, X-OAuth-Scopes, X-Accepted-OAuth-Scopes, X-Poll-Interval, X-GitHub-Media-Type, X-GitHub-SSO, X-GitHub-Request-Id, Deprecation, Sunset', 'Access-Control-Allow-Origin': '*', 'Strict-Transport-Security': 'max-age=31536000; includeSubdomains; preload', 'X-Frame-Options': 'deny', 'X-Co

## Query String Parameters

In [23]:
import requests

# Search GitHub's repositories for popular Python projects
response = requests.get(
    "https://api.github.com/search/repositories",
    params={"q": "language:python", "sort": "stars", "order": "desc"},)
    # params=b"q=language:python&sort=stars&order=desc",
'''
    requests.get(
...     "https://api.github.com/search/repositories",
...     [("q", "language:python"), ("sort", "stars"), ("order", "desc")]
)
'''

# Inspect some attributes of the first three repositories
json_response = response.json()
popular_repositories = json_response["items"]
for repo in popular_repositories[:3]:

    print(f"Name: {repo['name']}")
    print(f"Description: {repo['description']}")
    print(f"Stars: {repo['stargazers_count']}")
    print(repo)

Name: public-apis
Description: A collective list of free APIs
Stars: 295223
{'id': 54346799, 'node_id': 'MDEwOlJlcG9zaXRvcnk1NDM0Njc5OQ==', 'name': 'public-apis', 'full_name': 'public-apis/public-apis', 'private': False, 'owner': {'login': 'public-apis', 'id': 51121562, 'node_id': 'MDEyOk9yZ2FuaXphdGlvbjUxMTIxNTYy', 'avatar_url': 'https://avatars.githubusercontent.com/u/51121562?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/public-apis', 'html_url': 'https://github.com/public-apis', 'followers_url': 'https://api.github.com/users/public-apis/followers', 'following_url': 'https://api.github.com/users/public-apis/following{/other_user}', 'gists_url': 'https://api.github.com/users/public-apis/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/public-apis/starred{/owner}{/repo}', 'subscriptions_url': 'https://api.github.com/users/public-apis/subscriptions', 'organizations_url': 'https://api.github.com/users/public-apis/orgs', 'repos_url': 'https://api.github.com/

## Authentication

In [24]:
import requests
response = requests.get(
    "https://httpbin.org/basic-auth/user/passwd",
    auth=("user", "passwd")
)

response.status_code
response.request.headers["Authorization"]

'Basic dXNlcjpwYXNzd2Q='

from requests.auth import HTTPBasicAuth
requests.get(
"https://httpbin.org/basic-auth/user/passwd",
auth=HTTPBasicAuth("user", "passwd")
)


### implement token by a subclass of AuthBase
using class to add the token to request headers is better.

In [25]:
from requests.auth import AuthBase

class TokenAuth(AuthBase):
    """Implements a token authentication scheme."""

    def __init__(self, token):
        self.token = token

    def __call__(self, request):
        """Attach an API token to the Authorization header."""
        request.headers["Authorization"] = f"Bearer {self.token}"
        return request

In [26]:
response = requests.get(
"https://api.github.com/user",
auth=TokenAuth(yourtoken)
)

NameError: name 'yourtoken' is not defined

## SSL Certificate Verification